In [ ]:
from pyvis.network import Network
import networkx as nx
import pandas as pd
import sqlite3
import panel as pn

# Criando a conexão com o banco

In [ ]:
conn = sqlite3.connect("nba_salary.sqlite")
cur = conn.cursor()

### Vendo as tabelas

In [ ]:
tb = pd.read_sql("""
SELECT a.Player, a.Tm, a.G, b.season17_18
FROM Seasons_Stats as a
INNER JOIN NBA_season1718_salary as b
ON a.PLAYER = b.Player and a.Tm = b.Tm
WHERE a.Tm != "TOT" and a.Year = 2017
""", conn)
pd.set_option('display.max_columns', None)
tb.head()

In [ ]:
tb = pd.read_sql("""
SELECT *
FROM Seasons_Stats
WHERE Tm != "TOT" and Year = 2017
""", conn)
tb.head(5)

In [ ]:
df = pd.read_sql("""
SELECT a.`Player`, a.Tm, b.season17_18
FROM Seasons_Stats as a
INNER JOIN NBA_season1718_salary as b
ON a.Player = b.`Player`
WHERE a.Tm != "TOT"
""", conn)
pd.set_option('display.max_rows', None)
df.drop_duplicates(["Player", "Tm"], inplace = True)
df.sort_values("Player", inplace = True)
df.head(10)

In [ ]:
df["opa"] = df["Tm"] + df["season17_18"].astype(str)

In [ ]:
df.head(10)

In [ ]:
tb = pd.read_sql("""
SELECT Player, Tm, PTS
FROM Seasons_Stats
WHERE Tm != "TOT" and Year = 2017
""", conn)
tb.head()

In [ ]:
nba_net = Network(notebook = True, height = "750px", width = "100%", bgcolor = "#222222", font_color = "white")

# set the physics layout of the network
nba_net.force_atlas_2based()


sources = tb['Player']
targets = tb['Tm']
weights = tb['PTS']

edge_data = zip(sources, targets, weights)

for e in edge_data:
    src = e[0]
    dst = e[1]
    w = e[2]

    nba_net.add_node(src, src, title = src, group = dst, value = 3*w)
    nba_net.add_node(dst, dst, title = dst, shape = "image", image = "img2.png", value = 100*w)
    nba_net.add_edge(src, dst, value = 10*w)

neighbor_map = nba_net.get_adj_list()


# add neighbor data to node hover data
for node in nba_net.nodes:
    node["title"] += " {}<br>".format(len(neighbor_map[node["id"]])) + " Conexões:<br>" + "<br>".join(neighbor_map[node["id"]])
    node["value"] = len(neighbor_map[node["id"]]) # this value attrribute for the node affects node size


nba_net.show_buttons(filter_ = None)
nba_net.show("final1.html")

In [ ]:
nba_net = Network(notebook = True, height = "750px", width = "100%", bgcolor = None, font_color = "white")

# set the physics layout of the network
nba_net.force_atlas_2based()


sources = tb['Player']
targets = tb['Tm']
weights = tb['PTS']

edge_data = zip(sources, targets, weights)

for e in edge_data:
    src = e[0]
    dst = e[1]
    w = e[2]

    nba_net.add_node(src, src, title = src, group = dst, value = 3*w)
    nba_net.add_node(dst, dst, title = dst, color = "#d95e22", value = 100*w)
    nba_net.add_edge(src, dst, value = 10*w)

neighbor_map = nba_net.get_adj_list()


# add neighbor data to node hover data
for node in nba_net.nodes:
    node["title"] += " {}<br>".format(len(neighbor_map[node["id"]])) + " Conexões:<br>" + "<br>".join(neighbor_map[node["id"]])
    node["value"] = len(neighbor_map[node["id"]]) # this value attrribute for the node affects node size


nba_net.show("final1.png")

In [6]:
df_top10 = pd.read_sql("""
SELECT Player, Tm, season17_18
FROM NBA_season1718_salary
WHERE Tm != "TOT"
ORDER BY season17_18 DESC
LIMIT 10
""", conn)

nba_net = Network(notebook = True, height = "750px", width = "100%", bgcolor = "#222222", font_color = "white")

# set the physics layout of the network
nba_net.force_atlas_2based()


sources = df_top10['Player']
targets = df_top10['Tm']
weights = df_top10['season17_18']

edge_data = zip(sources, targets, weights)

for e in edge_data:
    src = e[0]
    dst = e[1]
    w = e[2]
##########




##########
    nba_net.add_node(src, src, title = src, shape = "image", image = "images/img2.png", group = dst, value = w)
    nba_net.add_node(dst, dst, title = dst, shape = "star", group = dst, value = w)
    nba_net.add_edge(src, dst, value = w)

neighbor_map = nba_net.get_adj_list()

# add neighbor data to node hover data
for node in nba_net.nodes:
    node["title"] += " {}<br>".format(len(neighbor_map[node["id"]])) + " Conexões:<br>" + "<br>".join(neighbor_map[node["id"]])
    #node["value"] = len(neighbor_map[node["id"]]) # this value attrribute for the node affects node size


nba_net.show("salarios.html")

In [8]:
nba = pd.read_sql("""
SELECT Player, Tm, Year
FROM Seasons_Stats
WHERE Tm != "TOT"
""", conn)
nba.head()

,Player,Tm,Year
0,Curly Armstrong,FTW,1950.0
1,Cliff Barker,INO,1950.0
2,Leo Barnhorst,CHS,1950.0
3,Ed Bartels,DNN,1950.0
4,Ed Bartels,NYK,1950.0


In [9]:
nba1989 = nba[nba["Year"] == 1989 ]
nba1990 = nba[nba["Year"] == 1990 ]
nba1991 = nba[nba["Year"] == 1991 ]

nba1989_CHI = nba1989[nba1989["Tm"] == "CHI"]
nba1990_CHI = nba1990[nba1990["Tm"] == "CHI"]
nba1991_CHI = nba1991[nba1991["Tm"] == "CHI"]

nba1989_CHI["P_ano"] = nba1989["Player"] + " " + nba1989["Year"].astype(str)
nba1989_CHI["T_ano"] = nba1989["Tm"] + " " + nba1989["Year"].astype(str)
nba1990_CHI["P_ano"] = nba1990["Player"] + " " + nba1990["Year"].astype(str)
nba1990_CHI["T_ano"] = nba1990["Tm"] + " " + nba1990["Year"].astype(str)
nba1991_CHI["P_ano"] = nba1991["Player"] + " " + nba1991["Year"].astype(str)
nba1991_CHI["T_ano"] = nba1991["Tm"] + " " + nba1991["Year"].astype(str)

In [ ]:
nba["P_ano"] = nba["Player"] + nba["Year"].astype(str)
nba["T_ano"] = nba["Tm"] + nba["Year"].astype(str)

got_net = Network(notebook = True, height = "750px", width = "100%", bgcolor = "#222222", font_color = "white")

# set the physics layout of the network
got_net.force_atlas_2based()


sources = nba['P_ano']
targets = nba['T_ano']
# weights = nba['season17_18']

edge_data = zip(sources, targets)

for e in edge_data:
    src = e[0]
    dst = e[1]
#   w = e[2]

    got_net.add_node(src, src, title=src)
    got_net.add_node(dst, dst, title=dst)
    got_net.add_edge(src, dst)

neighbor_map = got_net.get_adj_list()

# add neighbor data to node hover data
#for node in got_net.nodes:
#    node["title"] += " {}<br>".format(len(neighbor_map[node["id"]])) + " Conexões:<br>" + "<br>".join(neighbor_map[node["id"]])
#    node["value"] = len(neighbor_map[node["id"]]) # this value attrribute for the node affects node size
print(neighbor_map)


#got_net.show("example.html")

In [10]:
nba_net = Network(notebook=True, height="750px", width="100%", bgcolor="#222222", font_color="white")

# set the physics layout of the network
nba_net.force_atlas_2based()

got_data = nba1989_CHI

sources = got_data['P_ano']
targets = got_data['T_ano']
#weights = got_data['Weight']

edge_data = zip(sources, targets)

for e in edge_data:
    src = e[0]
    dst = e[1]

    nba_net.add_node(src, src, title=src, color = "#FA3113")
    nba_net.add_node(dst, dst, title=dst, shape = "image", image = "cb.png",)
    nba_net.add_edge(src, dst)


neighbor_map = nba_net.get_adj_list()

# add neighbor data to node hover data

for node in nba_net.nodes:
    node["title"] += " {}<br>".format(len(neighbor_map[node["id"]])) + " Conexões:<br>" + "<br>".join(neighbor_map[node["id"]])
    node["value"] = len(neighbor_map[node["id"]]) # this value attrribute for the node affects node size

got_data = nba1990_CHI

sources = got_data['P_ano']
targets = got_data['T_ano']
#weights = got_data['Weight']

edge_data = zip(sources, targets)

for e in edge_data:
    src = e[0]
    dst = e[1]

    nba_net.add_node(src, src, title=src, color = "red")
    nba_net.add_node(dst, dst, title=dst, shape = "image", image = "cb.png",)
    nba_net.add_edge(src, dst)


neighbor_map = nba_net.get_adj_list()

# add neighbor data to node hover data

for node in nba_net.nodes:
    node["title"] += " {}<br>".format(len(neighbor_map[node["id"]])) + " Conexões:<br>" + "<br>".join(neighbor_map[node["id"]])
    node["value"] = len(neighbor_map[node["id"]]) # this value attrribute for the node affects node size
    
    
    
got_data = nba1991_CHI

sources = got_data['P_ano']
targets = got_data['T_ano']
#weights = got_data['Weight']

edge_data = zip(sources, targets)

for e in edge_data:
    src = e[0]
    dst = e[1]

    nba_net.add_node(src, src, title=src, color = "#FA512E")
    nba_net.add_node(dst, dst, title=dst, shape = "image", image = "../images/cb.png",)
    nba_net.add_edge(src, dst)


neighbor_map = nba_net.get_adj_list()

# add neighbor data to node hover data

for node in nba_net.nodes:
    node["title"] += " {}<br>".format(len(neighbor_map[node["id"]])) + " Conexões:<br>" + "<br>".join(neighbor_map[node["id"]])
    node["value"] = len(neighbor_map[node["id"]]) # this value attrribute for the node affects node size
    

nba_net.show("chicago.html")

In [ ]:
tb = pd.read_sql("""
SELECT Player, Tm, PTS
FROM Seasons_Stats
WHERE Tm != "TOT" and Year = 2017
""", conn)
tb.head()

In [ ]:
got_data = nba

sources = got_data['P_ano']
targets = got_data['T_ano']
#weights = got_data['Weight']

edge_data = zip(sources, targets)

for e in edge_data:
    src = e[0]
    dst = e[1]

    got_net.add_node(src, src, title=src)
    got_net.add_node(dst, dst, title=dst, color = "green")
    got_net.add_edge(src, dst)


neighbor_map = got_net.get_adj_list()

# add neighbor data to node hover data


for node in got_net.nodes:
    if len(neighbor_map[node["id"]]) > 20:
        print(node["id"])